In [3]:
import librosa
import tensorflow as tf
import numpy as np
import os
import random

2023-11-13 19:34:40.621277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
labels_path = "/Users/zuriobozuwa/Downloads/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt"

dataset_path = "/Users/zuriobozuwa/Downloads/LA/ASVspoof2019_LA_train/flac"

In [17]:
N_MELS=91 #Number of mel bands

In [25]:
#Extract labels into a dictionary
labels = {}

with open(labels_path, 'r') as labels_file:
    label_lines = labels_file.readlines()

#Optional: reduce the amount of data that we have
#Useful for debugging
random.shuffle(label_lines)
fraction = 0.001
label_lines = label_lines[:int(len(label_lines)*fraction)]

#Labelling
for line in label_lines:
    parts = line.strip().split()
    filename = parts[1]
    label = 1 if parts[-1] == "bonafide" else 0
    labels[filename] = label

    

In [27]:
x = []
y = []

#progress printing
m = len(label_lines)
i = 1

for filename, label in labels.items():
    print(filename + " AYOOO " + str(label))
    filepath = os.path.join(dataset_path, filename + ".flac")
    #print(filepath)
    # Load the audio file
    audio, _ = librosa.load(filepath, duration=5)

    # mel-frequency spectogram
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, n_mels=N_MELS)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    max_time_steps = 150
    if mel_spectrogram.shape[1] < max_time_steps:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :max_time_steps]

    x.append(mel_spectrogram)
    y.append(label)
    
    print(str(i) + "/" + str(m), end="\r")
    i+=1

x = np.array(x)
y = np.array(y)


LA_T_6960015 AYOOO 0
LA_T_7297093 AYOOO 0
LA_T_3139970 AYOOO 0
LA_T_4964499 AYOOO 0
LA_T_5220491 AYOOO 0
LA_T_2705905 AYOOO 0
LA_T_9304117 AYOOO 0
LA_T_2357070 AYOOO 0
LA_T_2191846 AYOOO 0
LA_T_8803088 AYOOO 0
LA_T_4636566 AYOOO 0
LA_T_9984936 AYOOO 0
LA_T_4702763 AYOOO 0
LA_T_3886583 AYOOO 0
LA_T_4124955 AYOOO 1
LA_T_6744301 AYOOO 0
LA_T_2334320 AYOOO 0
LA_T_7153160 AYOOO 0
LA_T_9701027 AYOOO 0
LA_T_8936519 AYOOO 1
LA_T_2122771 AYOOO 0
LA_T_1540528 AYOOO 0
LA_T_1257953 AYOOO 0
LA_T_4927835 AYOOO 0
LA_T_4790840 AYOOO 0


In [28]:
y_encoded = tf.keras.utils.to_categorical(y,2)

In [29]:
print(y_encoded)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [16]:
x.shape

(25380, 91, 150)

In [17]:
len(y)

25380

In [18]:
#Split data into training and validation sets

split_index = int(0.8 * len(y))

x_train, x_val = x[:split_index], x[split_index:]
y_train, y_val = y_encoded[:split_index], y_encoded[split_index:]

In [5]:
input_shape = (N_MELS, 150, 1)
model_input = tf.keras.layers.Input(shape=input_shape)
model_input.shape

TensorShape([None, 91, 150, 1])

In [6]:
M = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(model_input)
M.shape

TensorShape([None, 89, 148, 32])

In [7]:
M = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(M)
M.shape

TensorShape([None, 44, 74, 32])

In [8]:
M = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(M)
M.shape

TensorShape([None, 42, 72, 64])

In [9]:
M = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(M)
M.shape

TensorShape([None, 21, 36, 64])

In [10]:
M = tf.keras.layers.Flatten()(M)
M.shape

TensorShape([None, 48384])

In [11]:
M = tf.keras.layers.Dense(units=128, activation='relu')(M)
M.shape

TensorShape([None, 128])

In [12]:
M = tf.keras.layers.Dropout(rate=0.5)(M)
M.shape

TensorShape([None, 128])

In [13]:
model_output = tf.keras.layers.Dense(units=2, activation='softmax')(M)

In [14]:
model = tf.keras.models.Model(inputs=model_input, outputs=model_output)

In [15]:
type(model)

keras.src.engine.functional.Functional

In [72]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [73]:
model.fit(x_train, y_train, epochs=22, validation_data=(x_val,y_val))
#TODO Batches??

2023-11-11 16:33:51.965597: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1108598400 exceeds 10% of free system memory.


Epoch 1/22
635/635 [==============================] - 24s 36ms/step - loss: 0.9419 - accuracy: 0.8982 - val_loss: 0.2173 - val_accuracy: 0.9066
Epoch 2/22
635/635 [==============================] - 21s 34ms/step - loss: 0.1963 - accuracy: 0.9189 - val_loss: 0.1880 - val_accuracy: 0.9332
Epoch 3/22
635/635 [==============================] - 21s 34ms/step - loss: 0.1493 - accuracy: 0.9427 - val_loss: 0.1295 - val_accuracy: 0.9539
Epoch 4/22
635/635 [==============================] - 22s 34ms/step - loss: 0.1139 - accuracy: 0.9564 - val_loss: 0.1111 - val_accuracy: 0.9590
Epoch 5/22
635/635 [==============================] - 22s 34ms/step - loss: 0.0820 - accuracy: 0.9684 - val_loss: 0.1426 - val_accuracy: 0.9464
Epoch 6/22
635/635 [==============================] - 22s 34ms/step - loss: 0.0690 - accuracy: 0.9737 - val_loss: 0.1156 - val_accuracy: 0.9679
Epoch 7/22
635/635 [==============================] - 21s 34ms/step - loss: 0.0514 - accuracy: 0.9812 - val_loss: 0.0803 - val_accuracy:

In [74]:
model.save("model-1.keras")